In [1]:
import pandas as pd
import numpy as np

C:\Users\Deutsch\AppData\Local\Temp\ipykernel_27440\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ndf = pd.read_csv('NIO.csv')

In [3]:
def estado(Cambio):
    if Cambio > 0:
        return 'Subio'
    elif Cambio < 0:
        return 'Bajo'
    else:
        return 'Estable'
    
ndf['Cambio'] = ndf['Close'].diff()

ndf['EstadoNuevo'] = ndf['Cambio'].apply(estado)

transicion = pd.crosstab(ndf['EstadoNuevo'].shift(), ndf['EstadoNuevo'], normalize=True)

nivelConfianza = 0.95

alpha = 1 - nivelConfianza #Seria 0.05

def chi2 (transicion):
    frecuencias = transicion.values
    fsum = transicion.sum(axis = 1)
    csum = transicion.sum(axis = 0)
    tsum = transicion.sum().sum()
    frecuencias_esperadas = np.outer(fsum, csum) / tsum
    estadistica_chi = np.sum((frecuencias - frecuencias_esperadas)**2/frecuencias_esperadas)
    grados = (transicion.shape[0]-1) * (transicion.shape[1]-1)
    return estadistica_chi, grados

valorCritico = 9.488
estadistica_chi, grados = chi2(transicion)
print(f"Estadistica Chi: {estadistica_chi}")
print(f"\nGrados de libertad: {grados}")
print(f"Teniendo en cuenta que nuestros grados de libertad son {grados} y tenemos un valor critico de {valorCritico} entonces:")
if estadistica_chi > valorCritico:
    print("Se rechaza")
else:
    print("No se rechaza")

for estado in transicion.index:
    suma_probabilidades = transicion.loc[estado].sum()
    print(f"Suma de probabilidades para el estado {estado}: {suma_probabilidades}")

for semanaActual in range(len(ndf)):
    actual = ndf['EstadoNuevo'].iloc[semanaActual]
    psiguiente = transicion.loc[actual]
    psiguienteN = psiguiente / psiguiente.sum()

    print(f"\nSemana {semanaActual + 1}")
    print(f"\nEstado actual - {actual}")
    print(f"\nProbabilidad siguiente: {psiguienteN}")

Estadistica Chi: 0.01251236024905052

Grados de libertad: 4
Teniendo en cuenta que nuestros grados de libertad son 4 y tenemos un valor critico de 9.488 entonces:
No se rechaza
Suma de probabilidades para el estado Bajo: 0.5192307692307692
Suma de probabilidades para el estado Estable: 0.007692307692307693
Suma de probabilidades para el estado Subio: 0.47307692307692306

Semana 1

Estado actual - Estable

Probabilidad siguiente: EstadoNuevo
Bajo       0.0
Estable    0.0
Subio      1.0
Name: Estable, dtype: float64

Semana 2

Estado actual - Subio

Probabilidad siguiente: EstadoNuevo
Bajo       0.536585
Estable    0.000000
Subio      0.463415
Name: Subio, dtype: float64

Semana 3

Estado actual - Subio

Probabilidad siguiente: EstadoNuevo
Bajo       0.536585
Estable    0.000000
Subio      0.463415
Name: Subio, dtype: float64

Semana 4

Estado actual - Bajo

Probabilidad siguiente: EstadoNuevo
Bajo       0.511111
Estable    0.007407
Subio      0.481481
Name: Bajo, dtype: float64

Semana 